In [25]:
print(2)

2


In [26]:
#to access files and folders
import os
#data analysis and manipulation library
import pandas as pd
#math operations for multi-dimensional arrays and matrices
import numpy as np
#visualziation libraries
# import matplotlib.pyplot as plt
# import seaborn as sns

In [27]:
DATA_DIR_PATH = 'CogLoad-1Hz/'
os.listdir(DATA_DIR_PATH)

['2nxs5_sensors.csv',
 'ibvx8_sensors.csv',
 '6frz4_sensors.csv',
 'f3j25_sensors.csv',
 '7swyk_sensors.csv',
 'iz3x1_sensors.csv',
 'd',
 '1mpau_sensors.csv',
 'c24ur_sensors.csv',
 'b7mrd_sensors.csv',
 'ef5rq_sensors.csv',
 'dkhty_sensors.csv',
 'rc1in_sensors.csv',
 'yljm5_sensors.csv',
 '8a1ep_sensors.csv',
 'iz2ps_sensors.csv',
 'tn4vl_sensors.csv',
 'e4gay_sensors.csv',
 'f1gjp_sensors.csv',
 'hpbxa_sensors.csv',
 '3caqi_sensors.csv',
 'bd47a_sensors.csv',
 'wjxci_sensors.csv',
 '5gpsc_sensors.csv']

In [28]:
files = os.listdir(DATA_DIR_PATH)

win_size = 30 #seconds

#lists to save 30second segments for each sensor
label_segments = []
temp_segments = []
hr_segments= []
gsr_segments= []
rr_segments= []
for file in files:
    if 'sensor' not in file:
      continue
    print()
    print('========')
    print('Reading data from:',file)
    df = pd.read_csv(DATA_DIR_PATH+file)
    print(df.shape)
    df = df.replace([np.inf, -np.inf], np.nan)
    df = df.dropna()
    # print('Shape of data',df.shape)

    #for each task get segments task_end - 30 seconds
    tasks = ['n2', 'n2_rest', 'n3', 'n3_rest', 'HP', 'HPrest', 'FA','FArest', 'GC', 'GCrest', 'NC', 'NCrest', 'SX', 'SXrest', 'PT','PTrest']

    for task in tasks:
        for level in df[df.task==task].level.unique():
            task_df = df[(df.task==task) & (df.level==level)]
            #print('Segments for task',task,level,task_df.task.unique())
            #print(task_df.head())
            #print(task_df.tail())
            #print(len(task_df))
            if len(task_df)>0:
                if 'rest' in task and 'n2' not in task and 'n3' not in task:
                    timestamp_end_arr = []
                    #print([i for i in task_df.timestamp.diff()])
                    timestamp_end_arr.extend(task_df[task_df.timestamp.diff()>10000].timestamp.values)
                    #print(task, timestamp_end_arr)
                    timestamp_end_arr.append(task_df.timestamp.iloc[-1])
                    #print(task,timestamp_end_arr)
                    for segment_end in timestamp_end_arr:

                        segment_sensor_data = task_df[task_df.timestamp<segment_end]
                        segment_sensor_data = segment_sensor_data.iloc[-win_size:]

                        #print('REST',segment_sensor_data.shape,segment_end)
                        if segment_sensor_data.shape[0]==win_size:
                            label_segments.append(segment_sensor_data.iloc[-1].values)
                            temp_segments.append(segment_sensor_data.temperature.values[:win_size])
                            hr_segments.append(segment_sensor_data.hr.values[:win_size])
                            gsr_segments.append(segment_sensor_data.gsr.values[:win_size])
                            rr_segments.append(segment_sensor_data.rr.values[:win_size])
                else:
                    segment_end = task_df.timestamp.iloc[-1]
                    segment_start= segment_end - win_size*1000
                    segment_sensor_data =task_df[(task_df.timestamp<=segment_end) & (task_df.timestamp>=segment_start)]
                    if 25 < segment_sensor_data.shape[0] < win_size:
                        print("sga", segment_start, segment_end)
                        #print('segment_sensor_data',segment_sensor_data.shape,'added 5 seconds')
                        segment_sensor_data =df[(df.timestamp<=segment_end) & (df.timestamp>=segment_start)]
                    if len(segment_sensor_data)>=win_size:
                        label_segments.append(segment_sensor_data.iloc[-1].values)
                        temp_segments.append(segment_sensor_data.temperature.values[:win_size])
                        hr_segments.append(segment_sensor_data.hr.values[:win_size])
                        gsr_segments.append(segment_sensor_data.gsr.values[:win_size])
                        rr_segments.append(segment_sensor_data.rr.values[:win_size])

    # break

label_segments = np.stack(label_segments)
temp_segments = np.stack(temp_segments)
hr_segments = np.stack(hr_segments)
gsr_segments = np.stack(gsr_segments)
rr_segments = np.stack(rr_segments)


print('Labels',label_segments.shape)
print('Temperature',temp_segments.shape)
print('Heartrate',hr_segments.shape)
print('GSR',gsr_segments.shape)
print('RR',rr_segments.shape)


Reading data from: 2nxs5_sensors.csv
(4237, 21)

Reading data from: ibvx8_sensors.csv
(4212, 21)

Reading data from: 6frz4_sensors.csv
(5433, 21)

Reading data from: f3j25_sensors.csv
(4843, 21)

Reading data from: 7swyk_sensors.csv
(4527, 21)

Reading data from: iz3x1_sensors.csv
(3963, 21)
sga 1507727628491 1507727658491
sga 1507728113487 1507728143487

Reading data from: 1mpau_sensors.csv
(3890, 21)
sga 1505834996504 1505835026504

Reading data from: c24ur_sensors.csv
(4339, 21)

Reading data from: b7mrd_sensors.csv
(5600, 21)

Reading data from: ef5rq_sensors.csv
(3931, 21)
sga 1504017980498 1504018010498

Reading data from: dkhty_sensors.csv
(3390, 21)

Reading data from: rc1in_sensors.csv
(4244, 21)
sga 1508230587506 1508230617506

Reading data from: yljm5_sensors.csv
(4176, 21)
sga 1508492134488 1508492164488

Reading data from: 8a1ep_sensors.csv
(2534, 21)

Reading data from: iz2ps_sensors.csv
(4679, 21)

Reading data from: tn4vl_sensors.csv
(3041, 21)

Reading data from: e4ga

In [29]:
#check the 30-second segments
print("Number of segments for each type of sensor data")
print('Labels',label_segments.shape)
print('Temperature',temp_segments.shape)
print('Heartrate',hr_segments.shape)
print('GSR',gsr_segments.shape)
print('RR',rr_segments.shape)

Number of segments for each type of sensor data
Labels (838, 21)
Temperature (838, 30)
Heartrate (838, 30)
GSR (838, 30)
RR (838, 30)


In [30]:
column_names = segment_sensor_data.columns.values
label_df = pd.DataFrame(label_segments,columns = column_names)

temp_df= pd.DataFrame(temp_segments)
hr_df= pd.DataFrame(hr_segments)
gsr_df= pd.DataFrame(gsr_segments)
rr_df= pd.DataFrame(rr_segments)

In [31]:
DATA_DIR_PATH = 'initial_data/'

print('Saving data to folder', DATA_DIR_PATH)
label_df.to_csv(DATA_DIR_PATH+'segments_labels.csv')
temp_df.to_csv(DATA_DIR_PATH+'segments_temperature.csv')
hr_df.to_csv(DATA_DIR_PATH+'segments_heartrate.csv')
gsr_df.to_csv(DATA_DIR_PATH+'segments_gsr.csv')
rr_df.to_csv(DATA_DIR_PATH+'segments_rr.csv')
print('Done')

Saving data to folder initial_data/
Done
